In [29]:
def RecursiveChange(money, coins):
    if money == 0:
        return 0
    minNumCoins = float("inf")
    for i in range(0, len(coins)):
        if money >= coins[i]:
            numCoins = RecursiveChange(money - coins[i], coins)
            if numCoins + 1 < minNumCoins:
                minNumCoins = numCoins + 1
    return minNumCoins

In [30]:
RecursiveChange(76, (5, 4, 1))

KeyboardInterrupt: 

Execution of the above function requires a lot of time, because it will loop through all the possible combinations of change and thus cause many repeated calculation

In [83]:
def DPchange(money, coins):
    minNumCoins = [None]*(money+1)
    minNumCoins[0] = 0
    for m in range(1, money + 1):
        minNumCoins[m] = float("inf")
        for i in range(0, len(coins)):
            if m >= coins[i]:
                if minNumCoins[m - coins[i]] + 1 < minNumCoins[m]:
                    minNumCoins[m] = minNumCoins[m - coins[i]] + 1
    print(minNumCoins)
    return minNumCoins[money]

In [84]:
DPchange(76, (5, 4, 1))

[0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16]


16

### Dynamic programming in an arbitrary DAG

### Backtracking in the Alignment Graph

In [1]:
import pandas as pd
import numpy as np
import copy

def LcsBackTrack(v, w):
    
    # index for s
    v_list = list(v)
    v_index = [v_list[i] + "_" + str(i + 1) for i in range(len(v_list))]
    v_index.insert(0, "v_Gap_0")
    
    # column names for s
    w_list = list(w)
    w_index = [w_list[i] + "_" + str(i + 1) for i in range(len(w_list))]
    w_index.insert(0, "w_Gap_0")
    
    s = pd.DataFrame(index=v_index, columns=w_index, data=-np.inf)
    #print(s)
    
    # initialize penalty score
    s.iloc[:,0] = np.arange(0, -len(v_list) - 1, -1)
    s.iloc[0,:] = np.arange(0, -len(v_list) - 1, -1)
    #s.iloc[0,0] = 0
    
    backTrack = copy.deepcopy(s)
    #print(s)
    
    def delta_score(m, n):
        if m == n:
            return 2
        else:
            return -1
    
    # recursion
    for i in range(1, len(v)+1):
        for j in range(1, len(w)+1):
            s.iloc[i, j] = max(s.iloc[i-1, j-1] + delta_score(v[i - 1], w[j - 1]), s.iloc[i-1,j] - 1, s.iloc[i, j-1] - 1)
            if s.iloc[i, j] == s.iloc[i-1, j-1] + delta_score(v[i - 1], w[j - 1]):
                backTrack.iloc[i, j] = "\\"
            elif s.iloc[i, j] == s.iloc[i-1,j] - 1:
                backTrack.iloc[i, j] = "|"  
            else: 
                backTrack.iloc[i, j] = "-"
    print(s)
    return backTrack
                                 

def OutputLCS(backTrack, v, w):
    v_align = []
    w_align = []
    i = len(v)
    print(i)
    j = len(w)
    print(j)
    while i != 0 and j != 0:
        if backTrack.iloc[i, j] == "\\":
            v_align.insert(0, v[i - 1])
            w_align.insert(0, w[j - 1])
            i = i - 1
            j = j - 1
        elif backTrack.iloc[i, j] == "|":
            v_align.insert(0, v[i - 1])
            w_align.insert(0, "_")
            i = i - 1
        elif backTrack.iloc[i, j] == "-":
            v_align.insert(0, "_")
            w_align.insert(0, w[j - 1])
            j = j - 1

    return v_align, w_align
                    

In [2]:
LcsBackTrack("ACAATCC", "AGCATGC")

         w_Gap_0  A_1  G_2  C_3  A_4  T_5  G_6  C_7
v_Gap_0        0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0
A_1           -1  2.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0
C_2           -2  1.0  1.0  3.0  2.0  1.0  0.0 -1.0
A_3           -3  0.0  0.0  2.0  5.0  4.0  3.0  2.0
A_4           -4 -1.0 -1.0  1.0  4.0  4.0  3.0  2.0
T_5           -5 -2.0 -2.0  0.0  3.0  6.0  5.0  4.0
C_6           -6 -3.0 -3.0  0.0  2.0  5.0  5.0  7.0
C_7           -7 -4.0 -4.0 -1.0  1.0  4.0  4.0  7.0


,w_Gap_0,A_1,G_2,C_3,A_4,T_5,G_6,C_7
v_Gap_0,0,-1,-2,-3,-4,-5,-6,-7
A_1,-1,\,-,-,\,-,-,-
C_2,-2,|,\,\,-,-,-,\
A_3,-3,\,\,|,\,-,-,-
A_4,-4,\,\,|,\,\,\,\
T_5,-5,|,\,|,|,\,-,-
C_6,-6,|,\,\,|,|,\,\
C_7,-7,|,\,\,|,|,\,\


In [156]:
OutputLCS(LcsBackTrack("ACAATCC", "AGCATGC"), "ACAATCC", "AGCATGC")

         w_Gap_0  A_1  G_2  C_3  A_4  T_5  G_6  C_7
v_Gap_0        0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0
A_1           -1  2.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0
C_2           -2  1.0  1.0  3.0  2.0  1.0  0.0 -1.0
A_3           -3  0.0  0.0  2.0  5.0  4.0  3.0  2.0
A_4           -4 -1.0 -1.0  1.0  4.0  4.0  3.0  2.0
T_5           -5 -2.0 -2.0  0.0  3.0  6.0  5.0  4.0
C_6           -6 -3.0 -3.0  0.0  2.0  5.0  5.0  7.0
C_7           -7 -4.0 -4.0 -1.0  1.0  4.0  4.0  7.0
7
7


(['A', '_', 'C', 'A', 'A', 'T', 'C', 'C'],
 ['A', 'G', 'C', '_', 'A', 'T', 'G', 'C'])